# Final.SQL

Заказчик - крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных. В ней содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Импорт библиотек и подключение к базе данных

In [15]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [16]:
# Задаём параметры подключения к БД
db_config = {'user': 'praktikum_student',         # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp',             # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', # адрес сервера
             'port': 6432,                        # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

In [17]:
# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

In [18]:
# Подключаемся к БД
engine = create_engine(connection_string)

## Выгрузка данных

In [19]:
# Создадим вспомогательную фукнцию для считывания всей таблицы
def get_data(name):
    query = '''
                SELECT * FROM {}
            '''.format(name)
    
    return pd.io.sql.read_sql(query, con = engine)

In [20]:
# Создадим словарь из таблиц
table_list=['books', 'authors', 'publishers', 'ratings', 'reviews']
table_dict = dict.fromkeys(table_list)

In [21]:
# Заполним свлоарь
for table in table_list:
    table_dict[table] = get_data(table)

In [36]:
table_dict['books'].head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [37]:
table_dict['books'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [23]:
table_dict['authors'].head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [38]:
table_dict['authors'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [24]:
table_dict['publishers'].head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [39]:
table_dict['publishers'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [25]:
table_dict['ratings'].head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [40]:
table_dict['ratings'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [26]:
table_dict['reviews'].head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [41]:
table_dict['reviews'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Все таблицы выгрузились корректно. Между собой они соотносятся по следующей схеме.


<img src="https://pictures.s3.yandex.net/resources/scheme_1589269096.png" alt="альтернативный текст">

## SQL запросы

In [27]:
# Создадим вспомогательную функцию для выполнения произвольных запросов
def make_query(query):
    return pd.io.sql.read_sql(query, con = engine)

### 1 запрос

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [28]:
query = '''
            SELECT COUNT(DISTINCT book_id)  FROM books
            WHERE 
            publication_date ::date >= '2000-01-01';
        '''
result = make_query(query)
result

,count
0,821


После 1 января 2000 года вышла 821 книга.

### 2 запрос

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [45]:
# Сделаем запрос и осортируем по количеству обзоров
query = '''
            SELECT 
                title as имя_книги,
                COUNT(DISTINCT reviews.review_id) as количество_обзоров,
                AVG( ratings.rating) as средний_рейтинг
            FROM books
            LEFT JOIN ratings ON ratings.book_id  = books.book_id
            LEFT JOIN reviews ON reviews.book_id  = books.book_id
            GROUP BY title
            ORDER BY  количество_обзоров DESC;
        '''
result = make_query(query)
result

,имя_книги,количество_обзоров,средний_рейтинг
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


### 3 запрос

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [17]:
query = ''' SELECT publisher as pub,
            COUNT(book_id) as cnt
            FROM books
            LEFT JOIN publishers ON publishers.publisher_id   = books.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY cnt DESC
            limit 1;
        '''
result = make_query(query)
result

,pub,cnt
0,Penguin Books,42


Это издательство - "Penguin Books"

### 4 запрос

Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками;

In [71]:
query = ''' SELECT 
                authors.author,
                AVG(ratings.rating) as avg_rate
            FROM books
            LEFT JOIN authors ON authors.author_id   = books.author_id
            LEFT JOIN ratings ON ratings.book_id  = books.book_id 
            WHERE books.title in 
            (SELECT 
                title
            FROM books
            LEFT JOIN ratings ON ratings.book_id  = books.book_id 
            GROUP BY title
            HAVING COUNT (DISTINCT rating_id) >= 50)
            GROUP BY authors.author
            ORDER BY avg_rate DESC
            limit 1;
        '''
result = make_query(query)
result

,author,avg_rate
0,J.K. Rowling/Mary GrandPré,4.287097


Получаем двух авторов: J.K. Rowling Mary GrandPré

### 5 запрос

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [48]:
# 1 способ
query = '''  SELECT AVG(SUBQ.cnt)
             FROM
                (SELECT 
                    ratings.username,
                    COUNT(DISTINCT review_id) as cnt
                FROM ratings
                LEFT JOIN reviews ON reviews.username   = ratings.username
                GROUP BY  ratings.username
                HAVING COUNT(DISTINCT rating_id) > 50) AS SUBQ;
        '''
result = make_query(query)
result

,avg
0,24.333333


In [58]:
# 2 способ
query = ''' SELECT AVG(SUBQ.cnt) 
            FROM
                (SELECT 
                    COUNT(DISTINCT review_id) as cnt
                 FROM reviews
                 WHERE reviews.username in 
                    (SELECT 
                           DISTINCT ratings.username
                    FROM ratings
                LEFT JOIN reviews ON reviews.username   = ratings.username
                GROUP BY  ratings.username
                HAVING COUNT(DISTINCT rating_id) > 50)
                GROUP BY  reviews.username) AS SUBQ;
        '''
result = make_query(query)
result

,avg
0,24.333333


В срденем такие пользователи делают по 17.5 обзоров

In [49]:
query = '''  SELECT 
                ratings.username,
                COUNT(DISTINCT review_id) as cnt
            FROM ratings
            LEFT JOIN reviews ON reviews.username   = ratings.username
            GROUP BY  ratings.username
            HAVING COUNT( DISTINCT rating_id) > 50;
        '''
result = make_query(query)
result

,username,cnt
0,jennifermiller,25
1,martinadam,27
2,paul88,22
3,richard89,26
4,sfitzgerald,28
5,xdavis,18
